# MONOCAMCCDRed a package to do CCD reduction for MONOCAM/LSST
===============================================================================================================

This is the notebook to do the whole CCD reduction pipeline using the **CCDPROC** python package from astropy astronomical tools.
We are considering raw data images that comes from fits file which header does not full conform to standard.
Some keywords are missing.

- Creation date Friday 2016 June 3rd
- Author Sylvie Dagoret-Campagne
- affiliation : LAL/IN2P3/CNRS

This notebook is inspired and adapted from the Matt Craig notebook available here
http://nbviewer.jupyter.org/gist/mwcraig/06060d789cc298bbb08e

## 1.) Required softwares

Please make sure you have this software installed before you begin (in addition to the usual scipy, numpy):

+ [astropy](http://astropy.org) v 0.4 or higher (install with: ``pip install astropy`` in a terminal/command window)
+ [ccdproc](http://ccdproc.readthedocs.org) v 0.1.1 or higher (install with: ``pip install ccdproc`` in a terminal/command window)
+ [msumastro](http://msumastro.readthedocs.org) v 0.5 or higher (install with : ``pip install msumastro`` in a terminal/command window)

## 2.) Main Packages imports

In [58]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

from astropy.modeling import models
from astropy import units as u
from astropy import nddata
from astropy.io import fits

import ccdproc
print 'ccdproc version',ccdproc.__version__

from astropy.modeling import models

ccdproc version 1.0.1


In [59]:
from scipy import stats  
import os
from datetime import datetime, timedelta

In [60]:
import bottleneck as bn  # numpy's masked median is slow...really slow (in version 1.8.1 and lower)
print 'bottleneck version',bn.__version__

bottleneck version 1.0.0


In [61]:
from msumastro import ImageFileCollection, TableTree
# I know you can't do correlated errors....
nddata.conf.warn_unsupported_correlated = False

In [62]:
now=datetime.utcnow()  # choose UTC time
datestr=str(now)
print 'standard date format for the analysis :',datestr
#  want the following format '2016-05-10T11:55:27.267'
date_of_analysis=now.strftime('%Y-%m-%dT%H:%M:%S')
print 'fits date format for the analysis : ',date_of_analysis

standard date format for the analysis : 2016-06-03 19:46:33.228110
fits date format for the analysis :  2016-06-03T19:46:33


## 3) Set some properties of the instrument that took these images

- MONOCAM, one of the 200 CCD plate of LSST camera.
- This CCD is 4Kx4K readout by 16 amplifiers

Eeach fits file contains a primary header plus the 16 images

- 16 images of size 544 x 2048 pixels
corresponding of :
- 8 columns of 544 pixels : 8 x 544 = 4352 pixels 
- 2 raws of 2048 pixels : 2 x 2048 = 4096 pixels

There is an excess of 4352-4096= 256 pixels along the columns.
It is possible there is 32 overscan slots.


A pre-look to the flats data using ds9 or python script allowed me to guess what are the Trim section and the Bias section

In [63]:
NB_OF_CHAN_AMPL=16    # 16 images in each of the fits file

# 4. Define a few convenience functions

All of these are really optional, but some are pretty convenient. They are provided in part to illustrate how one can combine the basic ``ccdproc`` commands into something more like a pipeline.

## a. Subtract overscan and trim images in a list

In [107]:
def oscan_and_trim(image_list):
    """
    Remove overscan and trim a list of images. The original list is replaced by a list of images
    with the changes applied.
    """
    for idx, img in enumerate(image_list):
        oscan = ccdproc.subtract_overscan(img,overscan=img[:,521:544], add_keyword={'oscan_sub': True, 'calstat': 'O'}, model=models.Polynomial1D(1))
        image_list[idx] = ccdproc.trim_image(oscan[:,10:521], add_keyword={'trimmed': True, 'calstat': 'OT'})

## b. Calculate fast medians (only really needed until numpy 1.9)

###     A bottleneck-based replacement for ma.median for a *single* array

As explained above, in numpy 1.8.1 and lower the masked median of a stack of images is very slow.

In [65]:
def bn_median(masked_array, axis=None):
    """
    Perform fast median on masked array
    
    Parameters
    ----------
    
    masked_array : `numpy.ma.masked_array`
        Array of which to find the median.
    
    axis : int, optional
        Axis along which to perform the median. Default is to find the median of
        the flattened array.
    """
    data = masked_array.filled(fill_value=np.NaN)
    med = bn.nanmedian(data, axis=axis)
    # construct a masked array result, setting the mask from any NaN entries
    return np.ma.array(med, mask=np.isnan(med))

### A bottleneck-based replacement for a stack (i.e. list) of masked arrays

By "stack" I mean a group of images, e.g. darks of the same exposure, for which the appropriate baseline image for identifying bad pixels is a median image (rather than a median or mean of the whole stack)

In [66]:
def avg_over_images(masked_arr, axis=0):
    """
    Calculate average pixel value along specified axis
    """
    return ma.mean(masked_arr, axis=axis)

def med_over_images(masked_arr, axis=0):
    """
    Calculate median pixel value along specified axis
    
    Uses bottleneck.nanmedian for speed
    """
    
    dat = masked_arr.data.copy()
    dat[masked_arr.mask] = np.NaN
    return bn.nanmedian(dat, axis=axis)

### A little function for displaying image statistics....

..which is useful for determining scale when displaying an image.

In [67]:
imstats = lambda dat: (dat.min(), dat.max(), dat.mean(), dat.std())

### To make the list of biases

In [68]:
def BuildFilelist(path,name,ext='.fits',start=1,stop=99):
    '''
    Make the list of filenames required by ccdproc
    
    input:
       path : path of files
       name : common root of bias filenames
       ext  : extension of filenames
       start,stop : indexes of files
    output:
       full filename list
    '''
    filelist = []
    for num in range(start,stop+1,1):
        strnum=biasnumberstr= '{0:02d}'.format(num)  # python >= 2.6
        filename=name+strnum+ext
        fullfilename=os.path.join(path,filename)
        filelist.append(fullfilename)
    return filelist

## 5.) Read data

In [69]:
path='/Users/dagoret-campagnesylvie/iraf/MonoCamMay2016/20160509'
ext_filename='.fits'

### 5.1) The biases

In [70]:
root_biasfilename='bias_'
bias_startnum=1
bias_stopnum=50

In [71]:
rawbias_list=BuildFilelist(path,root_biasfilename,start=bias_startnum,stop=bias_stopnum)

In [93]:
NB_OF_BIAS=len(rawbias_list)

### a. First, load the data as a list of ``CCDData`` objects.

If you don't need to modify the metadata you could use ``ccdproc.CCDData.from_hdu(hdu)`` to create the ``CCDData`` object.

In [110]:
allrawbias = []
for chan in range(1,NB_OF_CHAN_AMPL+1,1):
    ccd_chan = [ ccdproc.CCDData.read(image_file, hdu=chan,unit="adu") for image_file in rawbias_list ]
    allrawbias.append(ccd_chan)

### b. Subtract overscan and trim using convenience function

In [111]:
for chan in range(NB_OF_CHAN_AMPL):
    oscan_and_trim(allrawbias[chan])

### c. Combine biases using average

In [112]:
masterbias_list=[]
for chan in range(NB_OF_CHAN_AMPL):
    biases = ccdproc.Combiner(allrawbias[chan])
    master_bias = biases.average_combine()
    masterbias_list.append(master_bias)

### d. Make a pretty picture...

Because why wouldn't you?

In [123]:
print 'channel bias_min, bias_max, bias_mean, bias_std'
for chan in range(NB_OF_CHAN_AMPL):
    bias_min, bias_max, bias_mean, bias_std = imstats(np.asarray(masterbias_list[chan]))
    print chan,bias_min, bias_max, bias_mean, bias_std
    
    #plt.figure(figsize=(5, 5))
    #plt.imshow(masterbias_list[chan], vmax=bias_mean + 4*bias_std, vmin=bias_mean - 4*bias_std)

channel bias_min, bias_max, bias_mean, bias_std
0 -0.926953778281 16.3574736904 0.538108686844 0.314870975147
1 -0.639385815501 35.4088702041 0.835044293876 0.377250931763
2 -0.913389727384 38.8144456782 0.580852228386 0.311788112246
3 -0.890469859412 45.6265212098 0.430237770077 0.325260488652
4 -1.11275323819 21.4632614112 0.406213044674 0.343524256608
5 -0.963451023399 32.0665673545 0.380810888376 0.307063493072
6 -0.837847425025 55.5501415345 0.512449337236 0.307441787621
7 -0.893613608593 51.3755929349 0.498104392728 0.313701611048
8 -1.43605580358 30.6485440915 0.583935905825 0.411604744699
9 -1.33418415968 24.6522832673 0.339367110557 0.393719203177
10 -1.50468322554 684.242805897 0.437289507698 1.38351603217
11 -1.33862449849 68.966238715 0.615655893042 0.473045631306
12 -1.48307837304 76.331293991 0.354877731516 0.419033305126
13 -1.2807210196 75.5247703881 0.75011139443 0.407416808246
14 -2.42971475843 48.0989244106 0.631909913791 0.693127033129
15 -1.68870998362 168.55941076

### 5.2) The darks

In [ ]:
root_darkfilename='dark_'
dark_startnum=1
dark_stopnum=63

In [ ]:
rawdark_list=BuildFilelist(path,root_darkfilename,start=dark_startnum,stop=dark_stopnum)

In [ ]:
allrawdark = []
for chan in range(1,NB_OF_CHAN_AMPL+1,1):
    ccd_chan = [ ccdproc.CCDData.read(image_file, hdu=chan,unit="adu") for image_file in rawdark_list ]
    allrawdark.append(ccd_chan)


### 5.3) The sky flats

In [103]:
allrawbias[0][0].data.shape

(2048, 544)

# 6) Process bias

### b. Subtract overscan and trim using convenience function

In [108]:
for chan in range(NB_OF_CHAN_AMPL):
    oscan_and_trim(allrawbias[chan])

### c. Combine biases using average

In [82]:
import numpy as np
from astropy import units as u
arr1 = ccdproc.CCDData(np.ones([100, 100]), unit=u.adu)
type(arr1)

ccdproc.ccddata.CCDData

In [ ]:
no_scan = ccdproc.subtract_overscan(arr1, overscan=arr1[:, 90:100])
assert (no_scan.data == 0).all()

In [ ]:
no_scan = ccdproc.subtract_overscan(arr1, fits_section='[91:100, :]')
assert (no_scan.data == 0).all()

In [ ]:
add_keyword={'oscan_sub': True, 'calstat': 'O'}

In [ ]:
type(add_keyword)